## Redes Neurais Artificiais 2025.1

- **Disciplina**: Redes Neurais Artificiais 2025.1  
- **Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  
- **Github**: http://github.com/elloa  
        

Levando em conta a base de dados **_Forest Cover Type_**, esta segunda parte do Projeto Prático 2.2 diz respeito à proposição e avaliação de múltiplas redes neurais artificiais do tipo feedforward multilayer perceptron para o problema da classificação multi-classe da cobertura florestal em uma área do Roosevelt National Forest.

## Testando Redes Neurais sem os Atributos Categórios

1. Abra a base de dados em questão
2. Elimine todas as colunas relativas aos atributos categóricos
3. Armazene o atributo alvo em uma variável y e os atributos preditores em uma variável X
4. Efetue uma partição holdout 70/30 com o sklearn, distribuindo os exemplos de maneira aleatória
5. Efetue o escalonamento dos atributos

### Escalonando os atributos

O treinamento de uma rede neural artificial é mais eficiente quando os valores que lhes são fornecidos como entrada são pequenos, pois isto favorece a convergência. Isto é feito escalonando-se todos os atributos para o intervalo [0,1], mas precisa ser feito de maneira cautelosa, para que informações do conjunto de teste não sejam fornecidas no treinamento.

Há duas estratégias para tal escalonamento: normalização e padronização. Ambas possuem características particulares, vantagens e limitações, como é possível ver aqui: https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/


No nosso caso, vamos usar a padronização. Assim, com os atributos preditores do treinamento, isto é, X_train, deve-se subtrair a média e dividir pelo desvio padrão:

X_train_std = (X_train - np.mean(X_train))/np.std(X_train)

Em seguida, o mesmo deve ser feito com os atributos preditores do conjunto de testes, mas com padronização relativa ao conjunto de treinamento:

X_test_std = (X_test - np.mean(X_train))/np.std(X_train)

Se todo o conjunto X for utilizado na padronização, a rede neural receberá informações do conjunto de teste por meio da média e variância utilizada para preparar os dados de treinamento, o que não é desejável.

In [ ]:
## Reservado para a importação de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, ConfusionMatrixDisplay
import joblib
from joblib import Parallel, delayed

In [ ]:
# 1 - Abrindo a base de dados

df = pd.read_csv('covtype.csv')
df.head(10)

In [ ]:
# 2 - Removendo as colunas relacionadas aos atributos categóricos

atributos_categoricos = ['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
                        'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 
                        'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 
                        'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 
                        'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 
                        'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 
                        'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 
                        'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 
                        'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 
                        'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 
                        'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']

df.drop(columns = atributos_categoricos, inplace = True)

In [ ]:
df.columns

In [ ]:
# 3 - Armazenando o atributo alvo em uma variável y 
# e os atributos preditores em uma variável X

X, y = df.drop(columns = 'Cover_Type'), df['Cover_Type']
X

In [ ]:
# 4 e 5 - Criação de uma função para efetuar uma partição holdout 70/30
# e realizar a Padronização dos conjuntos de treino e teste:

def holdout_partition (X, y):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

    X_train_std = (X_train - np.mean(X_train))/np.std(X_train)

    X_test_std = (X_test - np.mean(X_train))/np.std(X_train)

    return X_train, X_test, y_train, y_test, X_train_std, X_test_std

In [ ]:
X_train, X_test, y_train, y_test, X_train_std, X_test_std = holdout_partition(X, y)

### Continuando

5. Treine uma rede neural multilayer perceptron para este problema com uma única camada e dez neurônios  
    5.1 Utilize a função de ativação ReLU  
    5.2 Utilize o solver Adam    
    5.3 Imprima o passo a passo do treinamento    
    5.4 Utilize o número máximo de épocas igual a 300  
6. Com o modelo em questão, após o treinamento, apresente:  
    6.1 Matriz de confusão para o conjunto de teste  
    6.2 Acurácia  
    6.3 F-Score  
    6.4 Precisão  
    6.5 Revocação  
7. Repita o treinamento da mesma rede anterior sem imprimir o passo a passo (verbose False) por 100 vezes  
    7.1 Cada uma destas repetições deve ser feita com uma nova partição Holdout  
    7.2 Apresente a média e o desvio padrão da acurácia e do F-Score para o conjunto de treino  
8. Repita por 100 vezes o treinamento desta mesma rede, mas utilizando o otimizador SGD  
    8.1 Apresente a média e o desvio padrão da acurácia e do F-Score para o conjunto de treino  
9. Houve influência da escolha do otimizador no desempenho da rede?

In [ ]:
# 5 - Criação da rede neural MultiLayer Perceptron (MLP):

mlp = MLPClassifier(
    hidden_layer_sizes = (10,),     # 1 camada com 10 neurônios
    activation = 'relu',            # função de ativação ReLU
    solver = 'adam',                # solver Adam
    max_iter = 300,                 # 300 épocas
    verbose = True,                 # passo a passo do treinamento
)

In [ ]:
# Treinamento da rede neural MLP:

mlp.fit(X_train_std, y_train)

y_pred = mlp.predict(X_test_std)

In [ ]:
# 6 - Matriz de Confusão

fig, ax = plt.subplots(figsize = (8, 6))

disp = ConfusionMatrixDisplay.from_predictions(y_test, y_pred, ax = ax, cmap = 'Greens')

ax.set_xlabel("Rótulos Preditos", fontsize = 10)
ax.set_ylabel("Rótulos Reais", fontsize = 10)

ax.tick_params(axis = 'both', labelsize = 8)

plt.show()

In [ ]:
# 6 - Acurácia, Precisão, Recall e F1-Score

metrics_data = {
    'Acurácia': [accuracy_score(y_test, y_pred)],
    'Precisão': [precision_score(y_test, y_pred, average = 'weighted', zero_division = 0)],
    'Recall': [recall_score(y_test, y_pred, average = 'weighted', zero_division = 0)],
    'F1-Score': [f1_score(y_test, y_pred, average = 'weighted', zero_division = 0)]
}

# Criar DataFrame e exibir em porcentagem
df_metricas = pd.DataFrame(metrics_data, index = ['MLP 1']).T * 100

# Formatar os valores para 2 casas decimais e adicionar '%'
df_metricas = df_metricas.applymap(lambda x: f"{x:.2f}%")

print(df_metricas)

In [ ]:
# 7 - Repetindo o treinamento da rede neural MLP anterior 100 vezes, com verbose = False :

accuracies = []
f1_scores = []

for i in range(100):
    X_train, X_test, y_train, y_test, X_train_std, y_test_std = holdout_partition(X, y)
    
    mlp = MLPClassifier(
    hidden_layer_sizes = (10,),     # 1 camada com 10 neurônios
    activation = 'relu',            # função de ativação ReLU
    solver = 'adam',                # solver Adam
    max_iter = 300,                 # 300 épocas
    verbose = False,                # sem passo a passo do treinamento
    )

    mlp.fit(X_train_std, y_train)
    y_pred = mlp.predict(X_train_std)
    
    accuracies.append(accuracy_score(y_train, y_pred))
    f1_scores.append(f1_score(y_train, y_pred, average = 'weighted', zero_division = 0))

In [ ]:
print("\nAdam (Treino)")
print(f"Acurácia - Média: {np.mean(accuracies):.4f}, Desvio Padrão: {np.std(accuracies):.4f}")
print(f"F1-Score - Média: {np.mean(f1_scores):.4f}, Desvio Padrão: {np.std(f1_scores):.4f}")

In [ ]:
# 8 - Repetindo o treinamento com solver = "sgd":

accuracies_sgd = []
f1_scores_sgd = []

for i in range(100):
    X_train, X_test, y_train, y_test, X_train_std, y_test_std = holdout_partition(X, y)
    
    mlp = MLPClassifier(
        hidden_layer_sizes = (10,),     # 1 camada com 10 neurônios
        activation = 'relu',            # função de ativação ReLU
        solver = 'sgd',                # solver Adam
        max_iter = 300,                 # 300 épocas
        verbose = False,                # sem passo a passo do treinamento
    )
    
    mlp.fit(X_train_std, y_train)
    y_pred = mlp.predict(X_train_std)

    accuracies_sgd.append(accuracy_score(y_train, y_pred))
    f1_scores_sgd.append(f1_score(y_train, y_pred, average = 'weighted', zero_division = 0))

In [ ]:
print("\nSGD (Treino)")
print(f"Acurácia - Média: {np.mean(accuracies_sgd):.4f}, Desvio Padrão: {np.std(accuracies_sgd):.4f}")
print(f"F1-Score - Média: {np.mean(f1_scores_sgd):.4f}, Desvio Padrão: {np.std(f1_scores_sgd):.4f}")

In [ ]:
if np.mean(accuracies) > np.mean(accuracies_sgd):
    print("\nAdam obteve melhor desempenho médio que SGD.")
else:
    print("\nSGD obteve melhor desempenho médio que Adam.")

# Diferença clara?
print(f"Diferença média de acurácia: {np.mean(accuracies) - np.mean(accuracies_sgd):.4f}")
print(f"Diferença média de F1-score: {np.mean(f1_scores) - np.mean(f1_scores_sgd):.4f}")

In [ ]:
print("Maior valor:", np.max(X))

## Discussão

Nos passos anteriores, você avaliou o desempenho de uma única rede neural que contém os seguintes parâmetros: uma única camada oculta com 10 neurônios e função de ativação ReLU. O otimizador utilizado, quer seja SGD ou ADAM, trata-se do algoritmo para aproximar o gradiente do erro. Neste sentido, a escolha do otimizador é um hiperparâmetro, pois diz respeito a como a rede neural definida previamente atuará "em tempo de execução"  durante o processo de treinamento. Também são hiperparâmetros a quantidade de épocas, a taxa de aprendizado inicial, dentre outros.

Cabe alientar também que você efetuou o treinamento desta rede por 100 vezes e apresentou os resultados em termos de média +- desvio padrão. Lembre-se que em uma rede neural há a inicialização aleatória de pesos e, em consequência, o desempenho delas está sujeito à uma flutuação estocástica. A execução destas múltiplas vezes faz com que eliminemos algum viés introduzido por uma boa ou má "sorte" na escolha de pesos no caso de uma única execução.

Você também aprendeu uma estratégia para escalonar os atributos para uma melhor convergência da rede. Utilize-a em todos os treinamentos e testes propostos a seguir.

## Propondo Novas Arquiteturas

Variando  os parâmetros (uma ou duas camadas ocultas, com diferente números de neurônios em cada uma delas e a função de ativação) e o hiperparâmetros solver (Adam ou SGD) e o número de épocas (100,150 e 200), atenda ao que se pede:

1. Proponha 10 arquiteturas distintas de RNAs para o problema em questão, à sua escolha
2. Avalie cada uma das arquiteturas perante todos os hiperparâmetros apresentados por 100 vezes
3. Como resultado da avaliação, apresente:  
    3.1 Top-3 melhores redes no tocante à F-Score e Acurácia  
    3.2 Repetição em que houve o melhor desempenho de cada uma dessas redes: ilustre tp, tf, fp e fn  

In [ ]:
# Arquiteturas propostas

arquiteturas = [((10,10), 'relu', 'adam', 100), ((23,27), 'tanh', 'sgd', 150), ((30,20), 'identity', 'adam', 200),
    ((40,), 'relu', 'adam', 100), ((33,), 'tanh', 'adam', 150), ((20,10), 'logistic', 'adam', 200),
    ((18,18), 'identity', 'adam', 100), ((35,10), 'relu', 'sgd', 150), ((10,10), 'identity', 'adam', 100),
    ((50,), 'identity', 'sgd', 100)
]


In [ ]:
# Função que retorna as 3 melhores arquiteturas com base no valor de acurácia e desempatando pelo valor de f1-score

def gerar_top3(media_geral_arquiteturas):
    top3_acurracy = sorted(media_geral_arquiteturas, key= lambda res: (res[1], res[0]), reverse= True)

    print("\n\n\nTop 3 resultados com maior acurácia:")
    for i in range(3):
        print(f"Top {i}: Arquitetura: {top3_acurracy[i][2]} Acurácia = {top3_acurracy[i][1]:.4f}, F1-Score = {top3_acurracy[i][0]:.4f} \n\n")
    return top3_acurracy[:3]



In [ ]:
# Função que gera os modelos de MLPClassifier com base na arquitetura passada como parâmetro, retornando os valores de acurácia de f1-score

def treino_e_avaliacao(X, y, arquitetura):
    X_train, X_test, y_train, y_test, X_train_std, X_test_std = holdout_partition(X, y)
    
    mlp = MLPClassifier(
        hidden_layer_sizes = arquitetura[0],
        activation = arquitetura[1],
        solver = arquitetura[2],
        max_iter = arquitetura[3],
        verbose = False
    )

    mlp.fit(X_train_std, y_train)
    y_pred = mlp.predict(X_test_std)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

    return acc, f1

In [ ]:

media_geral_arquiteturas = []  # Lista da media de acuracia, f1-score e indice de cada uma das arquiteturas

for j in range(len(arquiteturas)): 
    
    #print("Iteracao", j)
    
    # Paralelização das 100 iterações para cada uma das arquiteturas
    resultados = Parallel(n_jobs=-1)( #resultados vai ser uma lista de 100 valores de acuracia e f1
        delayed(treino_e_avaliacao)(X, y, arquiteturas[j]) for _ in range(100)
    )


    
    acuracias_arquitetura_i, f1scores_arquitetura_i  = zip(*resultados)

    resultados_arquitetura_i = (np.mean(f1scores_arquitetura_i), np.mean(acuracias_arquitetura_i), j) # Armazena a media das acuracias, f1scores e o indice da arquitetura
    media_geral_arquiteturas.append(resultados_arquitetura_i) # Armazena a média das métricas de cada arquitetura

In [ ]:
gerar_top3(media_geral_arquiteturas)

In [ ]:
# melhor_desempenho = sorted(media_geral_arquiteturas, key= lambda res: (res[1], res[0]), reverse= True)[0]
# print(melhor_desempenho)
# fig, ax = plt.subplots(figsize = (8, 6))

# disp = ConfusionMatrixDisplay.from_predictions(y_test, y_pred, ax = ax, cmap = 'Greens')

# ax.set_xlabel("Rótulos Preditos", fontsize = 10)
# ax.set_ylabel("Rótulos Reais", fontsize = 10)

# ax.tick_params(axis = 'both', labelsize = 8)

# plt.show()


## Estimando o número de neurônios

Um dos problemas de pesquisa com redes neurais artificiais consiste na determinação do número de neurônios em sua arquitetura. Embora não seja possível definir a priori qual rede neural é adequada para um problema, pois isto só é possível mediante uma busca exaustiva, há regras na literatura que sugerem o número de neurônios escondidos, tal como a regra da Pirâmide Geométrica, dada a seguir:

$$N_h = \alpha \cdot \sqrt{N_i \cdot N_o},$$

em que $N_h$ é o número de neurônios ocultos (a serem distribuídos em uma ou duas camadas ocultas), $N_i$ é o número de neurônios na camada de entrada e $N_o$ é o número de neurônios na camada de saída. 

1. Consulte a documentação da classe MLPClassifier (disponível em https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) e obtenha os valores de $N_i$ e $N_o$.
2. Teste os valores de $\alpha$ como sendo iguais a $0.5$, $2$ e $3$.
3. Proponha pelo menos 30 redes neurais segundo a regra da pirâmide geométrica e teste-as nos mesmos termos estabelecidos anterioremente  (solver, épocas, etc.)  
    3.1 Apresente as top-3 melhores redes no tocante à F-Score e Acurácia  

O valor de $N_i$ deve ser igual a quantidade de características de entrada do treinamento e o $N_0$ deve ser igual a quantidade de classes distintas, tedo em vista que este é um problema de classificação multiclasse. Assim:


In [ ]:
n_i = df.shape[1]
n_i

In [ ]:
n_0 = y.nunique()
n_0

In [ ]:
alphas = [0.5, 2, 3]

In [ ]:
# Função que atribui o valor de N_h como a nova descrição das camadas escondidas

def atualizar_camadas(nh, arquiteturas):
    novas_arqs = []
    for arq in arquiteturas:
        arq_list = list(arq)
        arq_list[0] = nh
        novas_arqs.append(tuple(arq_list))
    return novas_arqs

In [ ]:
# Função que cria uma nova lista de arquiteturas usando, agora, os valores de N_h como quantidade de neurônios na camada escondida

def criar_novas_arqs_com_nh(alphas, arquiteturas):

    novas_arqs=[]
    
    for alpha in alphas:
        
        n_h = int(alpha * np.sqrt(n_i * n_0)) # Calcula o valor de N_h

        for arq in arquiteturas:
            arq_list = list(arq)
            arq_list[0] = n_h
            novas_arqs.append(tuple(arq_list))
    return novas_arqs


In [ ]:
media_geral_nh=[] # Lista da media de acuracia, f1-score e indice de cada uma das arquiteturas


novas_arqs = criar_novas_arqs_com_nh(alphas, arquiteturas)
 
for j in range(len(novas_arqs)):  
    

    # Paralelização das 100 iterações para cada uma das arquiteturas
    resultados = Parallel(n_jobs=-1)(
        delayed(treino_e_avaliacao)(X, y, novas_arqs[j]) for _ in range(100)
    )

    acuracias_arquitetura_i_nh, f1scores_arquitetura_i_nh  = zip(*resultados)

    resultados_arquitetura_i_nh = (np.mean(f1scores_arquitetura_i_nh), np.mean(acuracias_arquitetura_i_nh), j) # Armazena a media das acuracias, f1scores e o indice da arquitetura

    media_geral_nh.append(resultados_arquitetura_i_nh)
        



In [ ]:
teste2_top3 = gerar_top3(media_geral_nh)

## Testando as Redes Neurais com Atributos Categóricos

1. Considere as 6 redes neurais obtidas nos dois top-3 anteriores (arquiteturas próprias e regra da pirâmide geométrica)
2. Com todos os atributos preditores da base de dados original, incluindo os categóricos, treine e teste estas mesmas redes por 100 repetições  
    2.1 Considere o melhor otimizador para cada uma delas  
    2.2 Faça uso de 200 épocas para treinamento  
    2.2 Apresente os resultados de acurácia e F-Score em termos da média +- dp para cada arquitetura
3. Apresente o gráfico boxplot para o F-Score das 6 arquiteturas perante as 100 repetições

In [ ]:

top3_v1 = gerar_top3(media_geral_arquiteturas) # Lista que guarda o top3 da primeira iteração

top3_v2 = gerar_top3(media_geral_nh) # Lista que guarda o top3 das arquiteturas com N_h

top6=[] 

# Armazena a configuração das arquiteturas com base nos indices obtidos do top3
top1, top2, top3 = arquiteturas[top3_v1[0][2]], arquiteturas[top3_v1[1][2]], arquiteturas[top3_v1[2][2]] 
top1_nh, top2_nh, top3_nh= novas_arqs[top3_v2[0][2]], novas_arqs[top3_v2[1][2]], novas_arqs[top3_v2[2][2]]

# Armazena os 2 top3 na lista top6
top6.append(top1)
top6.append(top2)
top6.append(top3)
top6.append(top1_nh)
top6.append(top2_nh)
top6.append(top3_nh)




In [ ]:
# Obter os dados sem os valores excluidos anteriormente
new_df = pd.read_csv('covtype.csv')
new_X, new_y = new_df.drop(columns = 'Cover_Type'), df['Cover_Type']

In [ ]:
# Nova funcao de teste com a variável max_iter predefinida para todos como 200

def treino_avaliacao_categoricos(X, y, arquitetura):
    X_train, X_test, y_train, y_test, X_train_std, X_test_std = holdout_partition(X, y)
    
    
    mlp = MLPClassifier(
        hidden_layer_sizes = arquitetura[0],
        activation = arquitetura[1],
        solver = arquitetura[2],
        max_iter = 200,
        verbose = False
    )

    mlp.fit(X_train_std, y_train)
    y_pred = mlp.predict(X_test_std)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

    return acc, f1

In [ ]:
new_medias_gerais=[]
for i in range (len(top6)):
    new_resultados = Parallel(n_jobs=-1)(
        delayed(treino_avaliacao_categoricos)(X, y, top6[i]) for _ in range (100)
    )

    new_acuracias_i, new_f1scores_i = zip(*new_resultados)

    new_resultados_arq = (np.mean(new_acuracias_i), np.mean(new_f1scores_i), i)
    new_medias_gerais.append(new_resultados_arq)
    print(f"\nARQUITETURA {i} Camadas: {top6[i][0]} Acuracia: {np.mean(new_acuracias_i):.4f} +- {np.std(new_acuracias_i):.4f} F1-Score: {np.mean(new_f1scores_i):.4f} +- {np.std(new_f1scores_i):.4f}\n")


## Considerações Parciais

1. É possível identificar uma rede com desempenho superior às demais?
2. Qual estratégia mostrou-se mais producente para a obtenção de boas arquiteturas (Estratégia Própria ou Pirâmide Geométrica)? Por quê?
3. Considerar os atributos categóricos trouxe melhorias? Justifique.
4. Um número maior de épocas trouxe melhorias?
5. Qual a maior dificuldade de resolução do problema proposto perante as RNAs?